In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [13]:
#Load Data and Perform some feature transformations
df=pd.read_csv("weatherdata.csv")
df.head()

,time,tavg,tmin,tmax,prcp,wspd,pres
0,2014-01-01,13.1,8.3,20.0,0.0,5.4,1019.1
1,2014-01-02,14.4,8.3,21.7,0.0,4.3,1017.9
2,2014-01-03,14.2,7.8,18.9,0.0,3.2,1015.7
3,2014-01-04,15.4,13.3,18.9,0.0,2.9,1015.6
4,2014-01-05,15.4,10.0,23.3,0.0,5.0,1018.8


In [14]:
df=df.drop(['tmin','tmax'],axis=1)
df.head()

,time,tavg,prcp,wspd,pres
0,2014-01-01,13.1,0.0,5.4,1019.1
1,2014-01-02,14.4,0.0,4.3,1017.9
2,2014-01-03,14.2,0.0,3.2,1015.7
3,2014-01-04,15.4,0.0,2.9,1015.6
4,2014-01-05,15.4,0.0,5.0,1018.8


In [15]:
df['time']=pd.to_datetime(df['time'])
df['month']=df['time'].dt.month
df.head()

,time,tavg,prcp,wspd,pres,month
0,2014-01-01,13.1,0.0,5.4,1019.1,1
1,2014-01-02,14.4,0.0,4.3,1017.9,1
2,2014-01-03,14.2,0.0,3.2,1015.7,1
3,2014-01-04,15.4,0.0,2.9,1015.6,1
4,2014-01-05,15.4,0.0,5.0,1018.8,1


In [16]:
#prepare data for the model
X=df[['month','prcp','wspd','pres']].values
y=df['tavg'].values

Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=20)

In [17]:
scaler=StandardScaler()
Xtrain=scaler.fit_transform(Xtrain)
Xtest=scaler.fit_transform(Xtest)

In [18]:
Xtrain=torch.tensor(Xtrain)
Xtest=torch.tensor(Xtest)
ytrain=torch.tensor(ytrain).view(-1,1)
ytest=torch.tensor(ytest).view(-1,1)

In [19]:
Xtrain=Xtrain.view(Xtrain.size(0),1,Xtrain.size(1))
Xtest=Xtest.view(Xtest.size(0),1,Xtest.size(1))


In [20]:
#Model Definition

class RNN(nn.Module):
    def __init__(self,inputsize,hiddensize,numlayers,outputsize):
        super(RNN,self).__init__()
        self.rnn=nn.RNN(inputsize,hiddensize,numlayers,batch_first=True)
        self.fc=nn.Linear(hiddensize,outputsize)

    def forward(self,x):
        h0=torch.zeros(numlayers,x.size(0),hiddensize).to(x.device)

        out,_=self.rnn(x,h0)
        out=self.fc(out[:,-1,:])

        return out
    
inputsize=4
hiddensize=64
numlayers=1
outputsize=1

model=RNN(inputsize,hiddensize,numlayers,outputsize)

criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [21]:
#training loop
numepochs=1000
for epoch in range(numepochs):
    model.train()
    optimizer.zero_grad()
    outputs=model(Xtrain)
    loss=criterion(outputs,ytrain)
    loss.backward()
    optimizer.step()
    print("epoch :",epoch,"loss :",loss.item())
    

epoch : 0 loss : 343.6366271972656
epoch : 1 loss : 343.04498291015625
epoch : 2 loss : 342.45147705078125
epoch : 3 loss : 341.8560791015625
epoch : 4 loss : 341.2588195800781
epoch : 5 loss : 340.6595764160156
epoch : 6 loss : 340.0583190917969
epoch : 7 loss : 339.4547424316406
epoch : 8 loss : 338.8487854003906
epoch : 9 loss : 338.24029541015625
epoch : 10 loss : 337.6289978027344
epoch : 11 loss : 337.0147705078125
epoch : 12 loss : 336.3973388671875
epoch : 13 loss : 335.7765808105469
epoch : 14 loss : 335.15216064453125
epoch : 15 loss : 334.5239562988281
epoch : 16 loss : 333.8916931152344
epoch : 17 loss : 333.2552490234375
epoch : 18 loss : 332.61431884765625
epoch : 19 loss : 331.96881103515625
epoch : 20 loss : 331.3183898925781
epoch : 21 loss : 330.6629943847656
epoch : 22 loss : 330.00225830078125
epoch : 23 loss : 329.33612060546875
epoch : 24 loss : 328.6642761230469
epoch : 25 loss : 327.986572265625
epoch : 26 loss : 327.30279541015625
epoch : 27 loss : 326.61279296

In [22]:
#test loop
model.eval()
with torch.no_grad():
    predictions=model(Xtest)
    testloss=criterion(predictions,ytest)
    print("loss :",testloss.item())

loss : 4.532889366149902
